In [92]:
import requests

In [93]:
API_PREFIX = 'http://localhost:8085'
WORKFLOW_ID = 'scoring'

Rule #1

In [94]:
payload = {
  "workflowId": WORKFLOW_ID,
  "id": "scoring-age",
  "variables": {
    "age": {
      "type": "NUMERIC"
    }
  },
  "when": "true",
  "then": "if age < 21 then 100 else if age > 60 then 100 else 200"
}

requests.post(url="{base}/rule/{workflow_id}".format(base=API_PREFIX, workflow_id=payload.get("workflowId")), json=payload)

<Response [200]>

Rule #2

In [95]:
payload = {
  "workflowId": WORKFLOW_ID,
  "id": "scoring-checks-last-six-months",
  "variables": {
    "score_requests": {
      "type": "NUMERIC"
    }
  },
  "when": "true",
  "then": "let {x: (100-score_requests*5);} if x < 0 then 0 else x"
}

requests.post(url="{base}/rule/{workflow_id}".format(base=API_PREFIX, workflow_id=payload.get("workflowId")), json=payload)

<Response [200]>

In [96]:
payload = {
  "workflowId": WORKFLOW_ID,
  "id": "scoring-overdue-debt",
  "variables": {
    "has_overdue_debt": {
      "type": "BOOLEAN"
    }
  },
  "when": "true",
  "then": "if has_overdue_debt then 0 else 100"
}

requests.post(url="{base}/rule/{workflow_id}".format(base=API_PREFIX, workflow_id=payload.get("workflowId")), json=payload)


<Response [200]>

In [106]:
payload = {
  "workflowId": WORKFLOW_ID,
  "id": "scoring-overdue-debt-over90days-last12months",
  "variables": {
    "has_overdue_debt_90days": {
      "type": "BOOLEAN"
    }
  },
  "when": "true",
  "then": "if has_overdue_debt_90days then 0 else 100"
}

requests.post(url="{base}/rule/{workflow_id}".format(base=API_PREFIX, workflow_id=payload.get("workflowId")), json=payload)

<Response [200]>

In [98]:
payload = {
  "workflowId": WORKFLOW_ID,
  "id": "scoring-max-parallel-credits-last12months",
  "variables": {
    "parallel_credits": {
      "type": "NUMERIC"
    }
  },
  "when": "true",
  "then": "let {x: parallel_credits*50;} if x>200 then 200 else x"
}

requests.post(url="{base}/rule/{workflow_id}".format(base=API_PREFIX, workflow_id=payload.get("workflowId")), json=payload)

<Response [200]>

In [102]:
payload = {
  "workflowId": WORKFLOW_ID,
  "id": "scoring-credit-score",
  "variables": {
    "scoring_age_result": {
      "rule_id": "scoring-age",
      "type": "NUMERIC"
    },
    "scoring_checks_last_six_months_result": {
      "rule_id": "scoring-checks-last-six-months",
      "type": "NUMERIC"
    }
  },
  "when": "true",
  "then": "scoring_age_result+scoring_checks_last_six_months_result"
}

requests.post(url="{base}/rule/{workflow_id}".format(base=API_PREFIX, workflow_id=payload.get("workflowId")), json=payload)

<Response [200]>

Rule test

In [107]:
payload = {
  "age": 61,
  "score_requests": 1,
  "has_overdue_debt": False,
  "has_overdue_debt_90days": False,
  "parallel_credits": 1
}

response = requests.post(url="{base}/eval/{workflow_id}".format(base=API_PREFIX, workflow_id=WORKFLOW_ID), json=payload)
response.text

'[{"ruleId":"scoring-checks-last-six-months","type":"NUMERIC","value":95,"fits":true},{"ruleId":"scoring-credit-score","type":"NUMERIC","value":195,"fits":true},{"ruleId":"scoring-age","type":"NUMERIC","value":100,"fits":true},{"ruleId":"scoring-max-parallel-credits-last12months","type":"NUMERIC","value":50,"fits":true},{"ruleId":"scoring-overdue-debt","type":"NUMERIC","value":100,"fits":true},{"ruleId":"scoring-overdue-debt-over90days-last12months","type":"NUMERIC","value":100,"fits":true}]'